
# Fractal Maps (ReLU)


In [ ]:

import sys, torch, platform
print("Python:", sys.version)
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPUs:", torch.cuda.device_count(), [torch.cuda.get_device_name(i) for i in range(torch.cuda.device_count())])


In [ ]:

from fractal_trainability import evaluate_grid, compose_rgb, save_rgb_image, select_device
import numpy as np

n = 16
steps = 300
width, height = 512, 512
eta0_range = (1e-3, 1e+1)   
eta1_range = (1e-3, 1e+3)   

device = select_device()  
res = evaluate_grid(n=n, steps=steps,
                    eta0_range=eta0_range, eta1_range=eta1_range,
                    grid=(height, width),
                    log_eta0=True, log_eta1=True,
                    seed=123, dtype64=True, device=device,
                    record_fractional_dim=True)

rgb = compose_rgb(res["diverged"], res["blue"], res["red"])
save_rgb_image("preview_map.png", rgb, dpi=100)
print("Wrote preview_map.png")



On 8×A100:

```bash
torchrun --standalone --nproc_per_node=8 fractal_trainability.py \
  --ddp --n 16 --steps 500 \
  --eta0-min 1e-3 --eta0-max 1e+1 \
  --eta1-min 1e-3 --eta1-max 1e+3 \
  --width 4096 --height 4096 \
  --out fractal_grid_4096x4096.npz
```


In [ ]:
import numpy as np
from fractal_trainability import compose_rgb, save_rgb_image

data = np.load("fractal_grid_4096x4096.npz", allow_pickle=True)
rgb = compose_rgb(data["diverged"], data["blue"], data["red"])
save_rgb_image("fractal_4096.png", rgb, dpi=100)

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

data = np.load("fractal_grid_4096x4096.npz", allow_pickle=True)  
frac = data["frac_dim"]  

plt.figure(figsize=(6,6))
im = plt.imshow(frac, origin="lower", interpolation="nearest")
plt.title("Mean fractional dimensionality (from $W_0$ columns)")
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.tight_layout()
plt.show()
